# Import

In [ ]:
!pip install pysad mat73
# colorama

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.0/69.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.6/200.6 kB 13.4 MB/s eta 0:00:00
  Created wheel for combo: filename=combo-0.1.3-py3-none-any.whl size=42855 sha256=bda1c5d73ecd553a17f8d77ff9288f3caa20b63067704c3b432da4fe51fc3b43
  Stored in directory: /root/.cache/pip/wheels/21/e7/9e/8baa8e183e9e3d7e37b74c5c0083462d91fb34123b92abb6ad
Successfully built combo


In [14]:
%%capture
import numpy as np
from numba import jit, objmode
from sklearn.utils import shuffle
import pandas as pd
from scipy.io import loadmat
import pandas as pd
import mat73
from scipy.stats import rankdata
from scipy.spatial.distance import cdist
import matplotlib.pyplot as plt
import time
from tqdm import tqdm
from sklearn.preprocessing import MinMaxScaler
import math
from datetime import datetime as dt

## module Show

In [ ]:
# AUC and AP metric
from sklearn.metrics import average_precision_score, roc_auc_score
def AUCAP(score,y):
  auc = roc_auc_score(y,score)
  ap = average_precision_score(y,score)
  print('auc = ',auc)
  print('ap = ' ,ap)
  return auc,ap

## Dataset

In [ ]:
# import data set
!git clone https://github.com/mossssd/Data.git

Cloning into 'Data'...
remote: Enumerating objects: 77, done.
remote: Total 77 (delta 0), reused 0 (delta 0), pack-reused 77 (from 1)
Receiving objects: 100% (77/77), 26.52 MiB | 11.45 MiB/s, done.
Resolving deltas: 100% (2/2), done.


#WMOF

In [ ]:
# Calculate number of points in radius
@jit(nopython=True)
def _point_in_radius(dm, sort_idx, win_size, n):
  idx = np.ones((win_size,n), dtype = np.int64)
  for i in range(win_size):
    pre_distance = -1
    point_count = n
    for j in range(n-1, -1, -1):
      obv_point = sort_idx[i][j]
      if dm[i][obv_point] != pre_distance:
        point_count = j
        pre_distance = dm[i][obv_point]
      idx[i][obv_point] = point_count
  return idx

In [ ]:
# Calculate mass ratio matrix
@jit(nopython=True)
def _Var_Massratio(Data,window):
  # Beware of large numbers, it might overflow python int
    n = len(Data)
    mass = np.zeros(n)
    mass2 = np.zeros(n)
    assert(window > 0)

    # slicing window through data
    for start_point in range(0,n,window):
      stop_point = min(start_point+ window, n)
      Current_Data = Data[start_point : stop_point]
      win_size = stop_point - start_point

      with objmode(window_dm = "i8[:, :]", sort_idx = "i8[:, :]"):
        window_dm = cdist(Current_Data, Data)
        sort_idx = np.argsort(window_dm)

      # when pairwise distance is same, the index is also same
      current_idx = _point_in_radius(window_dm, sort_idx, win_size, n)

      # calculate all current points
      for i in range(start_point, stop_point):
        for j in range(i+1, stop_point):
          m = (current_idx[j%window][i]*1.0 + 1)/ (current_idx[i%window][j] + 1)
          mass[i] += m
          mass2[i] += m**2
          mass[j] += 1/m
          mass2[j] += 1/m**2

      # calculate remaining points
      for i in range(stop_point,n):
        with objmode(dm = "i8[:, :]", sort_remain = "i8[:, :]"):
          dm = cdist([Data[i]], Data)
          sort_remain = np.argsort(dm)

        # when pairwise distance is same, the index is also same
        idx = _point_in_radius(dm, sort_remain, 1, n)

        for j in range(start_point, stop_point):
          m = (current_idx[j%window][i]*1.0 + 1 )/ (idx[0][j] + 1)
          mass[i] += m
          mass2[i] += m**2
          mass[j] += 1/m
          mass2[j] += 1/m**2

    var = mass2/(n-1)-(mass/(n-1))**2
    return var

In [ ]:
# Calculate windowing mass ratio
@jit(nopython=True)
def _Window_Massratio(Data,Window,Overlap_ratio):
  n =Data.shape[0]
  scores = np.zeros(n)

  assert(Window > 0 )
  assert(0.0 <= Overlap_ratio <= 0.5)

  overlap_size = int(Overlap_ratio * Window)
  mid_overlap_point = int(Window - overlap_size/2)
  score_count = 0

  # Windowing data and send to MOF
  for start_point in range(0, n , Window - overlap_size):
    stop_point = min(n, start_point + Window)
    w = stop_point - start_point
    current_data = Data[start_point : stop_point]
    # Last data window size (w) <= Window
    current_scores = _Var_Massratio(current_data,w)

    # Assign score to data points
    stop_score_count = min(n, start_point + mid_overlap_point)
    for i in range(score_count, stop_score_count):
      scores[i] = current_scores[i - start_point]
      score_count += 1

  return scores

In [ ]:
# Calculate stream windowing mass ratio
@jit(nopython=True)
def _Stream_Window_Massratio(Data, Window, Overlap_ratio, Start_idx):
  n =Data.shape[0]
  scores = np.zeros(n)

  assert(Window > 0 )
  assert(0.0 <= Overlap_ratio <= 0.5)

  mid_overlap_point = int((1 - Overlap_ratio/2) * Window)

  scores = _Var_Massratio(Data,Window)

  return mid_overlap_point - int((1 - Overlap_ratio) * Window), Window - int((1 - Overlap_ratio) * Window), scores[Start_idx : mid_overlap_point] # start_idx, len, scores

## Class

In [ ]:
# from adbench.myutils import Utils
class WMOF:

# Windowing mass-ratio-variance based outlier factor (WMOF)
# This algorithm is an extension of the mass-ratio-variance outlier factor algorithm (MOF).
# WMOF operates on overlapping windows of fixed size, specified by the user.
# The use of overlapping windows ensures that anomalies occurring at window boundaries are not missed.
# For each window, the MOF score is computed for all data points within the window.

# Parameters-free
# ----------
  def __init__(self, Window=1000, Overlap_ratio = 0.2, model_name:str=None):
    self.name='WMOF'
    self.Data = []
    self.DataCount = 0 # for fit_score
    self.start_index = 0 # for fit_score
    self.window_size = Window # for fit_score
    self.overlap_ratio = Overlap_ratio # for fit_score
    self.decision_scores_ = np.array([], dtype = np.float64)
    self.Anomaly = np.array([])

    #for pyod
    # self.utils = Utils()

  def fit(self,X_train):

    '''
    Parameters
    ----------
    Data : numpy array of shape (n_samples, n_features)
        The input samples.
    Window : integer (int)
        number of points for each calculation
        default window size is 1000.
    Overlap_ratio : float
        0.0 <= Overlap_ratio <= 0.5
        A Overlap_ratio between window frame.
        default ratio is 0.2
    '''
    '''
    Returns
    -------
    self : object
    '''

  #  Fitted estimator.
    Data = X_train
    self.Data = Data

  # Calculate Windowing mass ratio variance (WMOF)
    #self.decision_scores_= _Window_Massratio(Data,Window,Overlap_ratio)
    return self

  def decision_function(self, X, Window =1000):
    X = np.array(X)
    self.decision_scores_= _Window_Massratio(X,Window,0.2)
    return self.decision_scores_

  def fit_score(self, x):
    self.Data.append(x)
    self.DataCount += 1
    score = []
    if self.DataCount == self.window_size:

      self.start_index, self.DataCount, score =  _Stream_Window_Massratio(np.array(self.Data), self.window_size, self.overlap_ratio, self.start_index)
      self.Data = self.Data[int((1 - self.overlap_ratio) * self.window_size):]

    return score

  def fit_last_score(self):
    self.start_index, self.DataCount, score =  _Stream_Window_Massratio(np.array(self.Data), len(self.Data), 0, self.start_index)
    self.Data = self.Data[int((1 - self.overlap_ratio) * self.window_size):]

    return score

  def detectAnomaly(self, theshold):
    '''
    Parameters
    ----------
    theshold : float
        A theshold value for detect anomaly points
    '''
    '''
    Returns
    -------
    idx : numpy array of shape (n_samples,)
        An index array of anomaly points in data
    '''
    # Check data avaliablity
    assert(len(self.Data) != 0)

    if self.decision_scores_.shape[0] == 0:
      self.fit(self.Data)
    idx = np.squeeze(np.argwhere(self.decision_scores_ > theshold))
    self.Anomaly = np.append(self.Anomaly,idx).astype(np.int32)
    self.Anomaly = np.unique(self.Anomaly)

    return idx

  def detectStream(self, scores, tau = None, n = 0.01):
    '''
    Parameters
    ----------
    scores : numpy array of shape (N,)
        decision scores for the current window.
    tau : float
        A threshold value for detecting anomaly points.
        default tau is None, the threshold is determined by 'n'.
    n : float or int
        If float (0.01 <= n <= 0.49), it is the percentage of anomaly data points
        to select (e.g., 0.01 means the top 1% of scores).
        If int, it is the exact number of anomaly data points to select.
        default n is 0.01
    '''
    '''
    Returns
    -------
    idx : numpy array of shape (N,)
        An index array of anomaly points in the current window of data.
    '''
    # Check data availability
    assert(len(self.Data) != 0)

    # Ensure decision scores are available (by fitting if necessary)
    if len(scores) == 0:
      return np.array([])

    num_data_points = scores.shape[0]

    # 1. Check tau first
    if tau is not None:
      threshold = tau

    # 2. If tau is None, check n (integer or float)
    elif type(n) == int:
      k = min(n, num_data_points) # Number of anomalies to pick
      if k == 0:
          threshold = np.max(scores) + 1
      else:
          threshold = np.percentile(scores, 100 * (num_data_points - k) / num_data_points)

    elif type(n) == float:
      percentile_cutoff = 100 * (1.0 - n)
      if 100 < percentile_cutoff or percentile_cutoff <  50: # Check for safety if n > 0.5
          percentile_cutoff = 100

      threshold = np.percentile(scores, percentile_cutoff)

    else:
      threshold = np.max(scores) + 1 # Threshold that excludes all points

    idx = np.squeeze(np.argwhere(scores > threshold))

    if idx.ndim == 0:
        idx = np.array([idx])

    return idx

# Experiment

##Dataset

1. UCI machine learn
* AI4I 2020 Predictive Maintenance Dataset
2. Classic case
*    'mammography'
 *   'shuttle'
 *   'annthyroid'
 *   'satellite'
 *   'smtp'
 *   'cover'
  *  'mulcross'
  *  'pima'
  *  'breastw'
   * 'arrhythmia'
   * 'ionosphere'
3. Kaggle
* Controlled Anomalies Time Series https://www.kaggle.com/datasets/patrickfleith/controlled-anomalies-time-series-dataset
* Credit Card Fraud https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud



<!-- **Select only 1 dataset and go to PySad Section for testing** -->

In [ ]:
## List for Datasets
List_data_name_all = []
List_Xy_all = []

### UCI data

In [ ]:
pip install ucimlrepo

In [ ]:
from ucimlrepo import fetch_ucirepo

# fetch dataset
ai4i_2020_predictive_maintenance_dataset = fetch_ucirepo(id=601)
# AI4I 2020 Predictive Maintenance Dataset [Dataset]. (2020). UCI Machine Learning Repository. https://doi.org/10.24432/C5HS5C.

# data (as pandas dataframes)
X = ai4i_2020_predictive_maintenance_dataset.data.features
y = ai4i_2020_predictive_maintenance_dataset.data.targets

# metadata
print(y)

# variable information
print(ai4i_2020_predictive_maintenance_dataset.variables)
X

      Machine failure  TWF  HDF  PWF  OSF  RNF
0                   0    0    0    0    0    0
1                   0    0    0    0    0    0
2                   0    0    0    0    0    0
3                   0    0    0    0    0    0
4                   0    0    0    0    0    0
...               ...  ...  ...  ...  ...  ...
9995                0    0    0    0    0    0
9996                0    0    0    0    0    0
9997                0    0    0    0    0    0
9998                0    0    0    0    0    0
9999                0    0    0    0    0    0

[10000 rows x 6 columns]
                   name     role         type demographic description units  \
0                   UID       ID      Integer        None        None  None   
1            Product ID       ID  Categorical        None        None  None   
2                  Type  Feature  Categorical        None        None  None   
3       Air temperature  Feature   Continuous        None        None     K   
4   Process tem

,Type,Air temperature,Process temperature,Rotational speed,Torque,Tool wear
0,M,298.1,308.6,1551,42.8,0
1,L,298.2,308.7,1408,46.3,3
2,L,298.1,308.5,1498,49.4,5
3,L,298.2,308.6,1433,39.5,7
4,L,298.2,308.7,1408,40.0,9
...,...,...,...,...,...,...
9995,M,298.8,308.4,1604,29.5,14
9996,H,298.9,308.4,1632,31.8,17
9997,M,299.0,308.6,1645,33.4,22
9998,H,299.0,308.7,1408,48.5,25


In [ ]:
X_anomaly = X.replace('L', 1).replace('M', 2).replace('H', 3)
y_anomaly = y['Machine failure']
y_anomaly = y_anomaly.to_numpy()
y_anomaly2 = y['HDF'].to_numpy()
print(y_anomaly.shape)
X_all, y_all = np.array(X_anomaly),y_anomaly
# y_anomaly = np.array(y_anomaly.index.values)
# print(y_anomaly)

List_data_name_all.append("ai4i_2020")
List_Xy_all.append((X_all, y_all))

(10000,)


/tmp/ipython-input-3277770598.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X_anomaly = X.replace('L', 1).replace('M', 2).replace('H', 3)


### Classic UCI data

In [ ]:
# import data set to memory
def Data(name,s=0):
  if name ==  'http.mat' or name == 'smtp.mat':
    mat = mat73.loadmat('/content/Data/data/'+name)
  else:
    mat = loadmat('/content/Data/data/'+name)
  X_all = mat['X']
  y_all = mat['y'].ravel()
  if s==1:
    X_all, y_all = shuffle(X_all, y_all, random_state=42)
  return X_all, y_all

In [ ]:
# Classic dataset
data_list = [
    'mammography',
    'shuttle',
    'annthyroid',
    'satellite',
    'smtp',
    'cover',
    'mulcross',
    'pima',
    'breastw',
    'arrhythmia',
    'ionosphere',
]

for data_name in data_list:
  X_all , y_all = Data(data_name + ".mat", s=1) # Added s=1 to shuffle for cover

  List_data_name_all.append(data_name)
  List_Xy_all.append((X_all, y_all))

X_all.shape

In [ ]:
np.average(y_all)

np.float64(0.358974358974359)

###Kaggle data

In [17]:
import kagglehub

# Download selected version
path = kagglehub.dataset_download("patrickfleith/controlled-anomalies-time-series-dataset")

print("Path to dataset files:", path)


path = kagglehub.dataset_download("mlg-ulb/creditcardfraud")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'controlled-anomalies-time-series-dataset' dataset.
Path to dataset files: /kaggle/input/controlled-anomalies-time-series-dataset
Using Colab cache for faster access to the 'creditcardfraud' dataset.
Path to dataset files: /kaggle/input/creditcardfraud


In [ ]:
import pandas as pd

## cats dataset
df = pd.read_csv("/root/.cache/kagglehub/datasets/patrickfleith/controlled-anomalies-time-series-dataset/versions/2/data.csv")
sample = df[4700000:5000000].dropna(how='all')
# sample = df.dropna(how='all')
df_x = sample.drop(columns=['timestamp', 'y', 'category'])
df_y = sample['y']

X_all = df_x.to_numpy().astype("float64")
y_all = df_y.to_numpy().astype("int32")
print(sum(y_all)) # check amount of real anomaly
df_x.shape

List_data_name_all.append("cat")
List_Xy_all.append((X_all, y_all))

In [ ]:
## Credit card dataset
df = pd.read_csv("/kaggle/input/creditcardfraud/creditcard.csv")
sample = df#[:300000]
df_x = sample.drop(columns=['Time', 'Class'])
df_y = sample['Class']


X_all = df_x.to_numpy().astype("float64")
y_all = df_y.to_numpy().astype("int32")
print(sum(y_all)) # check amount of real anomaly
df_x.shape

List_data_name_all.append("creditcardfraud")
List_Xy_all.append((X_all, y_all))

492


In [ ]:
print(List_data_name_all)

In [ ]:
import pandas as pd

summary_data = []

for i in range(len(List_data_name_all)):
    data_name = List_data_name_all[i]
    X_data, y_labels = List_Xy_all[i]

    dimensions = X_data.shape[1]
    num_total_points = len(y_labels)
    num_anomaly = sum(y_labels)
    percentage_anomaly = (num_anomaly / num_total_points) * 100 if num_total_points > 0 else 0

    summary_data.append({
        "Dataset": data_name,
        "Dimensions": dimensions,
        "Total Points": num_total_points,
        "Number of Anomalies": num_anomaly,
        "Percentage of Anomalies (%)": f"{percentage_anomaly:.2f}"
    })

summary_df = pd.DataFrame(summary_data)
print(summary_df.to_markdown(index=False))

##PySAD

**Use command (ctrl + F10)** Call this cell and below

A output result is a csv file

Variables name for dataset **"X_all", "y"**

In [ ]:
# Import modules.
from sklearn.utils import shuffle
from pysad.evaluation import AUROCMetric, AUPRMetric

from pysad.utils import ArrayStreamer
from pysad.transform.postprocessing import RunningAveragePostprocessor
from pysad.transform.preprocessing import InstanceUnitNormScaler
from pysad.utils import Data
from tqdm import tqdm
import numpy as np

from time import time

from pyod.models.iforest import IForest
from sklearn.utils import shuffle
from pysad.evaluation import AUROCMetric
from pysad.models.integrations import ReferenceWindowModel
from pysad.utils import ArrayStreamer
from pysad.utils import Data
from tqdm import tqdm
import numpy as np

import copy

### classic model

In [ ]:
AUROC_list = []
AUPR_list = []
time_list = []
scores_all = []

In [ ]:
classic_window_size = 1000
classic_sliding_size = 500

In [ ]:
from pyod.models.iforest import IForest
from sklearn.utils import shuffle
from pysad.evaluation import AUROCMetric
from pysad.models.integrations import ReferenceWindowModel
from pysad.utils import ArrayStreamer
from pysad.utils import Data
from tqdm import tqdm
import numpy as np

In [ ]:
# Import modules.
from pyod.models.iforest import IForest
from sklearn.utils import shuffle
from pysad.evaluation import AUROCMetric
from pysad.models.integrations import ReferenceWindowModel


n_list = [
    1000,
          # 200,300,400,500,750
          ]


for n_number in n_list:

  AUROC_dict = {}
  AUPR_dict = {}
  time_dict = {}
  for i in range(len(List_data_name_all)):
    data_name = List_data_name_all[i]
    X_all, y_all = List_Xy_all[i]
    print(data_name)

    if __name__ == "__main__":
        np.random.seed(61)  # Fix seed.
        iterator = ArrayStreamer(shuffle=False)

        # Fit reference window integration to first 100 instances initially.
        model = ReferenceWindowModel(model_cls=IForest, window_size= classic_window_size, sliding_size=classic_sliding_size, initial_window_X=X_all[:100], n_estimators  = n_number)

        auroc = AUROCMetric()  # Init area under receiver-operating-characteristics curve metric tracker.
        aupr =  AUPRMetric()
        t0 = time()

        preprocessor = InstanceUnitNormScaler()
        scores = []

        duration = 0
        for X, y in tqdm(iterator.iter(X_all[100:], y_all[100:])):

            X = preprocessor.fit_transform_partial(X)

            t0 = time()
            model.fit_partial(X)  # Fit to the instance.
            score = model.score_partial(X)  # Score the instance.
            t1= time()
            duration += t1 - t0

            auroc.update(y, score)  # Update the metric.
            aupr.update(y, score)
            scores.append(score)

        # Output resulting AUROCS metric.
        print("AUROC: ", auroc.get())
        print("AUPR: ", aupr.get())
        print("time: ", duration)
        AUROC_dict[data_name] = auroc.get()
        AUPR_dict[data_name] = aupr.get()
        time_dict[data_name] = duration

        # scores_all.append(scores)
  AUROC_list.append(AUROC_dict)
  AUPR_list.append(AUPR_dict)
  time_list.append(time_dict)


ai4i_2020


9900it [02:02, 80.66it/s]


AUROC:  0.833071625241481
AUPR:  0.17749580923323346
time:  120.75289869308472
mammography


11083it [02:15, 82.04it/s]


AUROC:  0.6590684724127391
AUPR:  0.03642823717537012
time:  133.01651620864868
shuttle


...

AUROC:  0.7662643917325566
AUPR:  0.7230703484387794
time:  2.9835338592529297
cat


299900it [1:01:58, 80.65it/s]


AUROC:  0.5453740629085713
AUPR:  0.0405517089452818
time:  3659.5315227508545
creditcardfraud


284707it [58:23, 81.26it/s] 


AUROC:  0.7137222779788975
AUPR:  0.007457420039013779
time:  3446.7316496372223


In [ ]:
AUROC_list

[{'ai4i_2020': np.float64(0.833071625241481),
  'mammography': np.float64(0.6590684724127391),
  'shuttle': np.float64(0.9894418962846847),
  'annthyroid': np.float64(0.7516077834121917),
  'satellite': np.float64(0.7358543210115573),
  'smtp': np.float64(0.8121517619739158),
  'cover': np.float64(0.9430488452667586),
  'mulcross': np.float64(0.1571405311961988),
  'pima': np.float64(0.49908917247626927),
  'breastw': np.float64(0.601843062806518),
  'arrhythmia': np.float64(0.4885151763740771),
  'ionosphere': np.float64(0.7662643917325566),
  'cat': np.float64(0.5453740629085713),
  'creditcardfraud': np.float64(0.7137222779788975)}]

In [ ]:
# Import modules.
from pyod.models.lof import LOF

n_list = [
    1000,
          # 200,300,400,500,750
          ]
scores = []
# This example demonstrates the integration of a PyOD model via ReferenceWindowModel.
for n_number in n_list:
  AUROC_dict = {}
  AUPR_dict = {}
  time_dict = {}

  for i in range(len(List_data_name_all)):
    data_name = List_data_name_all[i]
    X_all, y_all = List_Xy_all[i]
    print(data_name)

    if __name__ == "__main__":
        np.random.seed(61)  # Fix seed.
        iterator = ArrayStreamer(shuffle=False)

        # Fit reference window integration to first 100 instances initially.
        model = ReferenceWindowModel(model_cls=LOF, window_size= classic_window_size, sliding_size=classic_sliding_size, initial_window_X=X_all[:100], n_neighbors= n_number)

        auroc = AUROCMetric()  # Init area under receiver-operating-characteristics curve metric tracker.
        aupr =  AUPRMetric()
        scores = []
        t0 = time()

        preprocessor = InstanceUnitNormScaler()

        duration = 0
        for X, y in tqdm(iterator.iter(X_all[100:], y_all[100:])):

            X = preprocessor.fit_transform_partial(X)

            t0 = time()
            model.fit_partial(X)  # Fit to the instance.
            score = model.score_partial(X)  # Score the instance.
            t1= time()
            duration += t1 - t0

            auroc.update(y, score)  # Update the metric.
            aupr.update(y, score)
            scores.append(score)

        # Output resulting AUROCS metric.

        print("AUROC: ", auroc.get())
        print("AUPR: ", aupr.get())
        print("time: ", duration)
        AUROC_dict[data_name] = auroc.get()
        AUPR_dict[data_name] = aupr.get()
        time_dict[data_name] = duration
        # scores_all.append(scores)

  AUROC_list.append(AUROC_dict)
  AUPR_list.append(AUPR_dict)
  time_list.append(time_dict)

ai4i_2020


9900it [00:10, 904.81it/s]


AUROC:  0.8275567729182848
AUPR:  0.19572142994501449
time:  10.517324209213257
mammography


11083it [00:12, 868.20it/s]


AUROC:  0.5593266476662947
AUPR:  0.025342186287219844
time:  12.334805965423584
shuttle


...

AUROC:  0.47815230961298383
AUPR:  0.4109212982878515
time:  0.286395788192749
cat


299900it [04:46, 1045.75it/s]


AUROC:  0.5752797838182979
AUPR:  0.06480778767451278
time:  274.1258225440979
creditcardfraud


284707it [04:43, 1003.61it/s]


AUROC:  0.6445396026625326
AUPR:  0.009330906186523859
time:  270.9491457939148


In [ ]:
AUROC_list

[{'ai4i_2020': np.float64(0.833071625241481),
  'mammography': np.float64(0.6590684724127391),
  'shuttle': np.float64(0.9894418962846847),
  'annthyroid': np.float64(0.7516077834121917),
  'satellite': np.float64(0.7358543210115573),
  'smtp': np.float64(0.8121517619739158),
  'cover': np.float64(0.9430488452667586),
  'mulcross': np.float64(0.1571405311961988),
  'pima': np.float64(0.49908917247626927),
  'breastw': np.float64(0.601843062806518),
  'arrhythmia': np.float64(0.4885151763740771),
  'ionosphere': np.float64(0.7662643917325566),
  'cat': np.float64(0.5453740629085713),
  'creditcardfraud': np.float64(0.7137222779788975)},
 {'ai4i_2020': np.float64(0.8275567729182848),
  'mammography': np.float64(0.5593266476662947),
  'shuttle': np.float64(0.9866915166307149),
  'annthyroid': np.float64(0.6628955377339683),
  'satellite': np.float64(0.7876658004682727),
  'smtp': np.float64(0.7944850181353875),
  'cover': np.float64(0.5397820004210051),
  'mulcross': np.float64(0.99863623

In [ ]:
# Import modules.
from pyod.models.ocsvm import OCSVM

# This example demonstrates the integration of a PyOD model via ReferenceWindowModel.
AUROC_dict = {}
AUPR_dict = {}
time_dict = {}

for i in range(len(List_data_name_all)):
  data_name = List_data_name_all[i]
  X_all, y_all = List_Xy_all[i]
  print(data_name)

  if __name__ == "__main__":
      np.random.seed(61)  # Fix seed.
      iterator = ArrayStreamer(shuffle=False)

      # Fit reference window integration to first 100 instances initially.
      model = ReferenceWindowModel(model_cls=OCSVM, window_size= classic_window_size , sliding_size=classic_sliding_size, initial_window_X=X_all[:100])

      auroc = AUROCMetric()  # Init area under receiver-operating-characteristics curve metric tracker.
      aupr =  AUPRMetric()
      scores = []


      preprocessor = InstanceUnitNormScaler()

      duration = 0
      for X, y in tqdm(iterator.iter(X_all[100:], y_all[100:])):

          X = preprocessor.fit_transform_partial(X)

          t0 = time()
          model.fit_partial(X)  # Fit to the instance.
          score = model.score_partial(X)  # Score the instance.
          t1= time()
          duration += t1 - t0

          auroc.update(y, score)  # Update the metric.
          aupr.update(y, score)
          scores.append(score)

      # Output resulting AUROCS metric.
      print("AUROC: ", auroc.get())
      print("AUPR: ", aupr.get())
      print("time: ", duration)
      AUROC_dict[data_name] = auroc.get()
      AUPR_dict[data_name] = aupr.get()
      time_dict[data_name] = duration

    # scores_all.append(scores)
AUROC_list.append(AUROC_dict)
AUPR_list.append(AUPR_dict)
time_list.append(time_dict)

ai4i_2020


9900it [00:04, 2172.55it/s]


AUROC:  0.8020263861504452
AUPR:  0.16430755801883684
time:  4.272564888000488
mammography


11083it [00:05, 1913.21it/s]


AUROC:  0.7186868776608221
AUPR:  0.06496051544717932
time:  5.475079298019409
shuttle


...

AUROC:  0.8820224719101124
AUPR:  0.8335220544344796
time:  0.10671353340148926
cat


299900it [02:30, 1991.46it/s]


AUROC:  0.56795836966767
AUPR:  0.08339108031298245
time:  141.48081016540527
creditcardfraud


284707it [02:32, 1861.14it/s]


AUROC:  0.7475500125935235
AUPR:  0.021610542121122053
time:  144.1424651145935


In [ ]:
# Import modules.
from pyod.models.knn import KNN

n_list = [
    1000,
          # 200,300,400,500,750
          ]

for n_number in n_list:

  AUROC_dict = {}
  AUPR_dict = {}
  time_dict = {}
  for i in range(len(List_data_name_all)):
    data_name = List_data_name_all[i]
    X_all, y_all = List_Xy_all[i]
    print(data_name)

    if __name__ == "__main__":
        np.random.seed(61)  # Fix seed.
        iterator = ArrayStreamer(shuffle=False)

        # Fit reference window integration to first 100 instances initially.
        model = ReferenceWindowModel(model_cls=KNN, window_size= classic_window_size, sliding_size=classic_sliding_size, initial_window_X=X_all[:200], n_neighbors= 100)

        auroc = AUROCMetric()  # Init area under receiver-operating-characteristics curve metric tracker.
        aupr =  AUPRMetric()
        scores = []

        preprocessor = InstanceUnitNormScaler()

        duration = 0
        for X, y in tqdm(iterator.iter(X_all[200:], y_all[200:])):

            X = preprocessor.fit_transform_partial(X)

            t0 = time()
            model.fit_partial(X)  # Fit to the instance.
            score = model.score_partial(X)  # Score the instance.
            t1= time()
            duration += t1 - t0

            auroc.update(y, score)  # Update the metric.
            aupr.update(y, score)
            scores.append(score)

        # Output resulting AUROCS metric.
        print("AUROC: ", auroc.get())
        print("AUPR: ", aupr.get())
        print("time: ", duration)
        AUROC_dict[data_name] = auroc.get()
        AUPR_dict[data_name] = aupr.get()
        time_dict[data_name] = duration

      # scores_all.append(scores)
  AUROC_list.append(AUROC_dict)
  AUPR_list.append(AUPR_dict)
  time_list.append(time_dict)

ai4i_2020


9800it [00:02, 3425.70it/s]


AUROC:  0.8160261451382208
AUPR:  0.10182337443239217
time:  2.6327435970306396
mammography


10983it [00:03, 3513.55it/s]


AUROC:  0.7355736204579919
AUPR:  0.045605549209835804
time:  2.8990063667297363
shuttle


...

AUROC:  0.9138257575757576
AUPR:  0.8682516433831918
time:  0.03456830978393555
cat


299800it [01:33, 3220.46it/s]


AUROC:  0.6182263255225442
AUPR:  0.0768650143932934
time:  85.54238986968994
creditcardfraud


284607it [01:41, 2799.74it/s]


AUROC:  0.7167719143270309
AUPR:  0.01975976351037196
time:  93.72251844406128


###Result

In [ ]:
print(AUROC_list)
# AUROC_list2.append(AUROC_list)

[{'ai4i_2020': np.float64(0.833071625241481), 'mammography': np.float64(0.6590684724127391), 'shuttle': np.float64(0.9894418962846847), 'annthyroid': np.float64(0.7516077834121917), 'satellite': np.float64(0.7358543210115573), 'smtp': np.float64(0.8121517619739158), 'cover': np.float64(0.9430488452667586), 'mulcross': np.float64(0.1571405311961988), 'pima': np.float64(0.49908917247626927), 'breastw': np.float64(0.601843062806518), 'arrhythmia': np.float64(0.4885151763740771), 'ionosphere': np.float64(0.7662643917325566), 'cat': np.float64(0.5453740629085713), 'creditcardfraud': np.float64(0.7137222779788975)}, {'ai4i_2020': np.float64(0.8275567729182848), 'mammography': np.float64(0.5593266476662947), 'shuttle': np.float64(0.9866915166307149), 'annthyroid': np.float64(0.6628955377339683), 'satellite': np.float64(0.7876658004682727), 'smtp': np.float64(0.7944850181353875), 'cover': np.float64(0.5397820004210051), 'mulcross': np.float64(0.9986362300711609), 'pima': np.float64(0.477160384

In [ ]:
print(AUPR_list)
# AUPR_list2.append(AUPR_list)

[{'ai4i_2020': np.float64(0.17749580923323346), 'mammography': np.float64(0.03642823717537012), 'shuttle': np.float64(0.7948227348610271), 'annthyroid': np.float64(0.18143041582470085), 'satellite': np.float64(0.5775647788209324), 'smtp': np.float64(0.07506631097195361), 'cover': np.float64(0.15476830344217146), 'mulcross': np.float64(0.055995551304894156), 'pima': np.float64(0.357566340325165), 'breastw': np.float64(0.43833328785951753), 'arrhythmia': np.float64(0.1716287632905301), 'ionosphere': np.float64(0.7230703484387794), 'cat': np.float64(0.0405517089452818), 'creditcardfraud': np.float64(0.007457420039013779)}, {'ai4i_2020': np.float64(0.19572142994501449), 'mammography': np.float64(0.025342186287219844), 'shuttle': np.float64(0.7904217287831585), 'annthyroid': np.float64(0.29697457687809153), 'satellite': np.float64(0.6688149273201507), 'smtp': np.float64(0.4242020806026413), 'cover': np.float64(0.010854184565985973), 'mulcross': np.float64(0.9877317230113337), 'pima': np.flo

In [ ]:
print(time_list)
# time_list2.append(time_list)

[{'ai4i_2020': 120.75289869308472, 'mammography': 133.01651620864868, 'shuttle': 596.5542016029358, 'annthyroid': 85.1617534160614, 'satellite': 74.88546013832092, 'smtp': 1179.0616390705109, 'cover': 3617.1235485076904, 'mulcross': 3273.133325815201, 'pima': 7.808765888214111, 'breastw': 7.620631456375122, 'arrhythmia': 4.233841180801392, 'ionosphere': 2.9835338592529297, 'cat': 3659.5315227508545, 'creditcardfraud': 3446.7316496372223}, {'ai4i_2020': 10.517324209213257, 'mammography': 12.334805965423584, 'shuttle': 53.03952360153198, 'annthyroid': 8.539700508117676, 'satellite': 5.81290340423584, 'smtp': 100.12802791595459, 'cover': 316.67832922935486, 'mulcross': 282.6055042743683, 'pima': 0.7510662078857422, 'breastw': 0.8511183261871338, 'arrhythmia': 0.4303274154663086, 'ionosphere': 0.286395788192749, 'cat': 274.1258225440979, 'creditcardfraud': 270.9491457939148}, {'ai4i_2020': 4.272564888000488, 'mammography': 5.475079298019409, 'shuttle': 22.91816782951355, 'annthyroid': 2.99

## WMOF model

In [ ]:
classifier_list = ["IForest","LOF","OCSVM",	"KNN"]

In [ ]:
auroc = AUROCMetric()  # Init area under receiver-operating-characteristics curve metric tracker.
aupr =  AUPRMetric()
scores = []
anomaly_list = np.array([])

window_size_list = [
                    1000,
                    # 500,
                    # 200, 100,
                    ]
for window_size in window_size_list:
  print("window_size: ", window_size)
  classifier_list.append("WMOF" + str(window_size))
  AUROC_dict = {}
  AUPR_dict = {}
  time_dict = {}
  iterator = ArrayStreamer(shuffle=False)

  for i in range(len(List_data_name_all)):
    data_name = List_data_name_all[i]
    X_all, y_all = List_Xy_all[i]
    print(data_name)

    model = WMOF(window_size, Overlap_ratio=0.10 )
    preprocessor = InstanceUnitNormScaler()

    scores_WMOF = np.array([], dtype = np.float64)
    duration = 0

    for X, y in tqdm(iterator.iter(X_all, y_all)):

        X = preprocessor.fit_transform_partial(X)
        t0 = time()
        score = model.fit_score(X)  # Score the instance.
        t1= time()
        duration += t1 - t0
        new_anomaly_list = model.detectStream(score)

        # WMOF updates data with batch method.
        scores_WMOF = np.append(scores_WMOF,score)

    t0 = time()
    score = model.fit_last_score()
    duration += time() - t0
    scores_WMOF = np.append(scores_WMOF,score)
    roc, ap = AUCAP(scores_WMOF, y_all)

    print("time: ", duration)
    AUROC_dict[data_name] = roc
    AUPR_dict[data_name] = ap
    time_dict[data_name] = duration

    # scores_all.append(scores)
  AUROC_list.append(AUROC_dict)
  AUPR_list.append(AUPR_dict)
  time_list.append(time_dict)


window_size:  1000
mammography


11183it [00:05, 2179.09it/s]


auc =  0.6331794590102746
ap =  0.02972832971376631
time:  5.427340984344482


###Result

In [ ]:
# AUROC_list2.append(AUROC_list)
# AUPR_list2.append(AUPR_list)
# time_list2.append(time_list)

print(len(AUROC_list), AUROC_list)
print(AUPR_list)
print(time_list)

1 [{'mammography': np.float64(0.6331794590102746)}]
[{'mammography': np.float64(0.02972832971376631)}]
[{'mammography': 5.427340984344482}]


In [ ]:
import pandas as pd
import numpy as np

# Create a DataFrame for AUROC (datasets as rows, classifiers as columns)
auroc_columns_data = {classifier_name: [AUROC_list[i][dataset_name] for dataset_name in List_data_name_all]
                      for i, classifier_name in enumerate(classifier_list)}
auroc_df = pd.DataFrame(auroc_columns_data, index=List_data_name_all)
print("AUROC DataFrame:")
display(auroc_df)
auroc_df.to_csv('auroc_results.csv')

# Create a DataFrame for AUPR (datasets as rows, classifiers as columns)
aupr_columns_data = {classifier_name: [AUPR_list[i][dataset_name] for dataset_name in List_data_name_all]
                     for i, classifier_name in enumerate(classifier_list)}
aupr_df = pd.DataFrame(aupr_columns_data, index=List_data_name_all)
print("\nAUPR DataFrame:")
display(aupr_df)
aupr_df.to_csv('aupr_results.csv')

# Create a DataFrame for Time (datasets as rows, classifiers as columns)
time_columns_data = {classifier_name: [time_list[i][dataset_name] for dataset_name in List_data_name_all]
                     for i, classifier_name in enumerate(classifier_list)}
time_df = pd.DataFrame(time_columns_data, index=List_data_name_all)
print("\nTime DataFrame:")
display(time_df)
time_df.to_csv('time_results.csv')

AUROC DataFrame:


,WMOF1000
mammography,0.633179



AUPR DataFrame:


,WMOF1000
mammography,0.029728



Time DataFrame:


,WMOF1000
mammography,5.427341
